In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse

In [ ]:
data_dir = '../data/'

In [ ]:
df = pd.read_csv(data_dir + 'feat_eng_join_clean_train_aggressiveness_comentarios_facebook.csv')

# Crear Everygram

In [ ]:
ngram_max = 2
count_vectorizer = CountVectorizer(ngram_range=(1,ngram_max))
X_vec = count_vectorizer.fit_transform(df['Text'])
vocab = count_vectorizer.get_feature_names()

X = pd.DataFrame(X_vec.toarray(), columns = vocab)
y = df.Category

In [ ]:
X['Longitud_Text'] = df['Longitud_Text']
X['Numero_Palabras_Text'] = df['Numero_Palabras_Text']
X['Numero_Palabras_Unicas'] = df['Numero_Palabras_Unicas']

In [ ]:
X_sparse = sparse.csr_matrix(X.values)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)

In [ ]:
rf = RandomForestClassifier(class_weight="balanced")

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
rf_f1_score = f1_score(y_pred, y_test, average='micro')

In [ ]:
rf_f1_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred, average='micro')

In [ ]:
recall_score(y_test, y_pred, average='micro')

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(class_weight="balanced")
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_estimator_

In [ ]:
rf = rf_random.best_estimator_
scores = cross_val_score(rf, X_train, y_train, cv=5, scoring="f1_micro")
y_train_predict = cross_val_predict(rf, X_train, y_train, cv=3)
print("scores of Random forest is :" , scores)

In [ ]:
confusion_matrix(y_train, y_train_predict)

In [ ]:
precision_score(y_train, y_train_predict, average="micro")

In [ ]:
recall_score(y_train, y_train_predict, average="micro")